In [1]:
#analyze new

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from Bio import SeqIO
from Bio.Seq import Seq

import warnings
warnings.simplefilter('ignore')

import itertools
import numpy as np

import tqdm
import gc

import scipy as sp

In [2]:
#read in all_data_df
all_data_df=pd.read_csv('data/all_data_df.csv')
all_data_df

,virus_background,replicate,selection,sample,barcode,bc_count
0,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAAAAAGCTCGAGCTTGGCCGTACAGT,3
1,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAAACGGCGATAGCTGAAAAATTCACG,1
2,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAACATGCCATAGCTGAAAACGATACT,1
3,USSR77,R1,NoAb,USSR77_R1_NoAb,ACAAAAAGCTCGAGCTACTGCAGGCGCT,2
4,USSR77,R1,NoAb,USSR77_R1_NoAb,ACAAAAAGCTCGAGCTATTAACAAGTCC,1
...,...,...,...,...,...,...
2755098,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGACATAGCTCCTGATCAGGTC,1
2755099,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTCGAAGTTTAGAG,1
2755100,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTGCGGTTACCGGC,3
2755101,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTGTATGTCGGCTT,1


In [3]:
#trim data to only include those with bc_count less than bc_count_min values
#note that here bc_count refers to the number of barcode reads that are associated with unique UMIs
bc_count_min = 1

all_data_df_trimmed = all_data_df[all_data_df['bc_count']>=bc_count_min]

counts_before_filter = len(all_data_df)
counts_after_filter = len(all_data_df_trimmed)
counts_removed = counts_before_filter - counts_after_filter

print(f'''Minimum barcode count has been set to: {bc_count_min}
{counts_removed} barcodes have been removed, {counts_after_filter} unique barcodes remain.
Here is a breakdown of the unique barcode reads found in each sample:''')

filtered_stats=pd.DataFrame()
uniques=all_data_df['sample'].unique()

sample_list=[]
unique_bcs_before_filter_list = []
unique_bcs_after_filter_list = []

for s in tqdm.tqdm(uniques):
    before = len(all_data_df[all_data_df['sample'] == s])
    after = len(all_data_df_trimmed[all_data_df_trimmed['sample'] == s])
    
    sample_list.append(s)
    unique_bcs_before_filter_list.append(before)
    unique_bcs_after_filter_list.append(after)

filtered_stats['sample'] = sample_list 
filtered_stats['unique_bcs_before_filter'] = unique_bcs_before_filter_list
filtered_stats['unique_bcs_after_filter'] = unique_bcs_after_filter_list
filtered_stats['unique_bcs_removed'] = filtered_stats['unique_bcs_before_filter'] - filtered_stats['unique_bcs_after_filter']

pd.set_option('display.max_rows',None)
filtered_stats

Minimum barcode count has been set to: 1
0 barcodes have been removed, 2755103 unique barcodes remain.
Here is a breakdown of the unique barcode reads found in each sample:


100%|██████████| 64/64 [00:11<00:00,  5.45it/s]


,sample,unique_bcs_before_filter,unique_bcs_after_filter,unique_bcs_removed
0,USSR77_R1_NoAb,129770,129770,0
1,USSR77_R2_NoAb,87882,87882,0
2,SN89_R1_NoAb,77658,77658,0
3,SN89_R2_NoAb,87633,87633,0
4,SI06_R1_NoAb,74699,74699,0
5,SI06_R2_NoAb,92546,92546,0
6,SYD21_R1_NoAb,73849,73849,0
7,SYD21_R2_NoAb,104515,104515,0
8,SN89_R1_UCA860,46226,46226,0
9,SN89_R2_UCA860,29208,29208,0


In [7]:
#load the barcode pacbio data
background_list=['USSR77','SN89','SI06','SYD21']
geno_to_pheno_list=[]
geno_to_pheno_df=pd.DataFrame()

for x in background_list:
    geno_to_pheno_list.append(f'data/codon_variant_tables/codon_variant_table_{x}.csv')

for x in tqdm.tqdm(geno_to_pheno_list):
    temp_df=pd.read_csv(x)
    geno_to_pheno_df=pd.concat([geno_to_pheno_df,temp_df])

geno_to_pheno_df=geno_to_pheno_df.rename(columns={'target':'virus_background'})
geno_to_pheno_df['replicate']='R'+geno_to_pheno_df['library'].str[-1:]

print('PacBio data has been loaded\nHere is a sample of the PacBio dataframe:')
pd.set_option('display.max_rows',10)
geno_to_pheno_df

100%|██████████| 4/4 [00:00<00:00,  4.63it/s]


PacBio data has been loaded
Here is a sample of the PacBio dataframe:


,virus_background,library,barcode,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions,replicate
0,USSR77,rep1,ACAACAGAGTGCAGCTAGACGACCAACC,1,CCA176ACA,P176T,1,1,R1
1,USSR77,rep1,ACAACTGTAAGTAGCTAGGCTGTGCGGC,4,GTA125CAA,V125Q,1,1,R1
2,USSR77,rep1,ACAAGATCGGTCAGCTCAGTATCTGAAT,4,GTG216GAT,V216D,1,1,R1
3,USSR77,rep1,ACAAGGATTTACAGCTCGGTTCACCCAG,3,AGC179TTC,S179F,1,1,R1
4,USSR77,rep1,ACAAGGATTTACAGCTTGGTTGCTACTC,1,AGC179TTC,S179F,1,1,R1
...,...,...,...,...,...,...,...,...,...
407607,SYD21,rep2,TTGCGGTCCATTAGCTGCGATCGCCCCC,1,AGG130CAT,R130H,1,1,R2
407608,SYD21,rep2,TTGCGTACATCCAGCTCGGATGGTGCCT,2,AAT142GGA,N142G,1,1,R2
407609,SYD21,rep2,TTGCTAAGTACCAGCTCTACCAAAATGC,1,GGA110AAT,G110N,1,1,R2
407610,SYD21,rep2,TTGCTCACGCTCAGCTAGATATACAATC,5,GGA80TGC,G80C,1,1,R2


In [8]:
pd.set_option('display.max_rows',10)
all_data_df_trimmed

,virus_background,replicate,selection,sample,barcode,bc_count
0,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAAAAAGCTCGAGCTTGGCCGTACAGT,3
1,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAAACGGCGATAGCTGAAAAATTCACG,1
2,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAACATGCCATAGCTGAAAACGATACT,1
3,USSR77,R1,NoAb,USSR77_R1_NoAb,ACAAAAAGCTCGAGCTACTGCAGGCGCT,2
4,USSR77,R1,NoAb,USSR77_R1_NoAb,ACAAAAAGCTCGAGCTATTAACAAGTCC,1
...,...,...,...,...,...,...
2755098,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGACATAGCTCCTGATCAGGTC,1
2755099,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTCGAAGTTTAGAG,1
2755100,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTGCGGTTACCGGC,3
2755101,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTGTATGTCGGCTT,1


In [9]:
#merge illumina data with pacbio data
all_data_df_merged=all_data_df_trimmed.merge(geno_to_pheno_df,on=['virus_background','replicate','barcode'])

all_data_dropped = all_data_df_trimmed.merge(all_data_df_merged, how='outer', indicator=True)
all_data_dropped = all_data_dropped[all_data_dropped['_merge'] == 'left_only']
all_data_dropped = all_data_dropped.drop(columns=['_merge'])

print('PacBio data has been loaded\nHere is a sample of the merged dataframe:')
all_data_df_merged

PacBio data has been loaded
Here is a sample of the merged dataframe:


,virus_background,replicate,selection,sample,barcode,bc_count,library,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions
0,USSR77,R1,NoAb,USSR77_R1_NoAb,ACAAGGATTTACAGCTTGGTTGCTACTC,340,rep1,1,AGC179TTC,S179F,1,1
1,USSR77,R1,Ab9207,USSR77_R1_Ab9207,ACAAGGATTTACAGCTTGGTTGCTACTC,24,rep1,1,AGC179TTC,S179F,1,1
2,USSR77,R1,NoAb,USSR77_R1_NoAb,ATCTCATGCCATAGCTCCTTGTGCCCCA,20,rep1,6,GAA189ATG,E189M,1,1
3,USSR77,R1,NoAb,USSR77_R1_NoAb,ATCTCCGTATATAGCTGTGATGGCTCAC,104,rep1,1,AAA71CCA,K71P,1,1
4,USSR77,R1,NoAb,USSR77_R1_NoAb,ATCTTCAGGTACAGCTTTACCGCTCCCC,240,rep1,3,TCA126GTG,S126V,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
326242,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,ATCTGAATTCTCAGCTTCCTCGTACGTA,325,rep2,6,CCC135GAA,P135E,1,1
326243,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,ATCTGTCGGGAAAGCTTACTAAGGAATA,1,rep2,6,GTG218ATA,V218I,1,1
326244,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,ATCTTATCCAGTAGCTGGCCTGTCTTGC,15,rep2,4,GCA17TCA CCA66GTG,A17S P66V,2,2
326245,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,ATCTTGCGTTACAGCTTGATCCGATCAA,136,rep2,4,CCG229GTG,P229V,1,1


In [10]:
all_data_df_merged.query("aa_substitutions == 'D140R' & selection == 'UCA652' ")

,virus_background,replicate,selection,sample,barcode,bc_count,library,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions
73852,SN89,R1,UCA652,SN89_R1_UCA652,TCTTAAGAACGTAGCTCGCCTTTCAGCC,871,rep1,6,GAC140AGA,D140R,1,1
73971,SN89,R1,UCA652,SN89_R1_UCA652,TCTTAAGAACGTAGCTTCAAGGTCGATC,6285,rep1,6,GAC140AGA,D140R,1,1
151703,SN89,R2,UCA652,SN89_R2_UCA652,TCTTAAGAACGTAGCTCGTATAGATAGA,1,rep2,6,GAC140AGA,D140R,1,1
151843,SN89,R2,UCA652,SN89_R2_UCA652,TCTTAAGAACGTAGCTTAAAGCGGTCCT,1,rep2,6,GAC140AGA,D140R,1,1


In [11]:
#output counts files for individual samples for loading in to the analyze_diffsel notebook

replicate_list=['R1','R2']
cols=['barcode','count','codon_substitutions','aa_substitutions','variant_call_support']
background_list=['USSR77','SN89','SI06','SYD21']
antibody_list=['NoAb','UCA860','CH65','CH65_ND','UCA652','H2227','H1244','H1825','H679','UCA641','H1270','H1261','Ab6649','Ab9207','ADI-77474','ADI-77470','ADI-86325']

for bkg in background_list:
    for rep in replicate_list:
        for ab in antibody_list:
        
            temp_df = all_data_df_merged.query(f"virus_background == '{bkg}' & replicate == '{rep}' & selection == '{ab}' & n_aa_substitutions <= 1 ").rename(columns={'bc_count':'count'})
        
            if len(temp_df) == 0:
                continue
                
            else:
                temp_df = temp_df[cols].reset_index(drop=True)
                temp_df = temp_df.sort_values(by = 'count', ascending=False)
        
                temp_df.to_csv(f'data/counts_files/{bkg}_{rep}_{ab}.csv')